# Predictions

I will use this notebook to input, process and create predictions on 2018 data. Next I will create a series of CSV files to use in a GIS analysis of the data.

## Table of Contents
* [Imports](#Imports)
* [2018 Data](#2018_Data)
   * [Predictions](#Predictions)
* [GIS Analysis Files](#GIS_Analysis_Files)

# Imports

In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler

import autoreload
%load_ext autoreload
%autoreload 2
import python_code.Scripts as sc
import python_code.Reference as ref

In [2]:
model= pickle.load(open('final_model.sav', 'rb'))

# 2018 Data

In [3]:
df = pd.read_csv('./SNAP_data/qc_pub_fy2018.csv')
df.head()

,FSAFIL1,FSAFIL2,FSAFIL3,FSAFIL4,FSAFIL5,FSAFIL6,FSAFIL7,FSAFIL8,FSAFIL9,FSAFIL10,...,WORK12,WORK13,WORK14,WORK15,WORK16,FSNONCIT,FSDIS,FSELDER,FSKID,STATENAME
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,0,1,0,Connecticut
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,1,0,0,Connecticut
2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,1,0,0,Connecticut
3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,Connecticut
4,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,1,0,0,Connecticut


In [4]:
df.shape

(43738, 814)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43738 entries, 0 to 43737
Columns: 814 entries, FSAFIL1 to STATENAME
dtypes: float64(373), int64(440), object(1)
memory usage: 271.6+ MB


### Predictions

In [6]:
df = sc.clean_data_to_predict(df)

In [7]:
ss= StandardScaler()
X = ss.fit_transform(df.drop(columns=['state']))
df['predict']=model.predict(X)
df

,state,CERTHHSZ,VEHICLEA,FSDIS,FSSTDDE2,FSGRINC,RAWNET,FSSSI,FSTOTDE2,FSUNEARN,...,FSEARN,FSWAGES,FSSLTDE2,RAWERND,WRK_POOR,FSNELDER,FSSLTDED,FSGA,FSNETINC,predict
0,Connecticut,1.0,1.0,0.0,240.0,920.0,176.0,0.0,744.0,920.0,...,0.0,0.0,584.0,0.0,0.0,1.0,584.0,0.0,176.0,0.0
1,Connecticut,1.0,1.0,1.0,240.0,1409.0,239.0,0.0,1171.0,1197.0,...,212.0,212.0,969.0,42.0,1.0,0.0,969.0,0.0,238.0,1.0
2,Connecticut,1.0,1.0,1.0,240.0,1278.0,149.0,0.0,1129.0,1278.0,...,0.0,0.0,969.0,0.0,0.0,0.0,969.0,0.0,149.0,1.0
3,Connecticut,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Connecticut,1.0,1.0,1.0,240.0,1113.0,151.0,735.0,962.0,1113.0,...,0.0,0.0,802.0,0.0,0.0,0.0,802.0,0.0,151.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43733,Guam,2.0,1.0,0.0,150.0,150.0,0.0,0.0,150.0,150.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43734,Guam,2.0,1.0,0.0,393.0,1076.0,468.0,0.0,608.0,0.0,...,1076.0,1076.0,72.0,215.0,1.0,0.0,72.0,0.0,468.0,1.0
43735,Guam,3.0,1.0,0.0,321.0,1031.0,504.0,0.0,527.0,0.0,...,1031.0,1031.0,0.0,206.0,1.0,0.0,0.0,0.0,504.0,1.0
43736,Guam,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
df.to_csv('./SNAP_data/2018_indicators/preds_all.csv',index=None)

In [9]:
pred_df = pd.DataFrame(df.groupby('state')['predict'].value_counts().unstack()).rename(columns={0.0:'No',1.0:'Yes'})

In [10]:
pred_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53 entries, Alabama to Wyoming
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   No      53 non-null     int64
 1   Yes     53 non-null     int64
dtypes: int64(2)
memory usage: 1.2+ KB


In [11]:
pred_df.to_csv('./SNAP_data/2018_indicators/preds_target.csv',index='state')

# GIS Analysis Files

These series of dataframes show:

coef_df = the model tree interpreter coefficients impact on the model prediction, sorted by largest to smallest impact.
top = the top 15 of the most impactful coefficients on prediction.
coef_2018 = a dataframe of the top most impactful coefficients retrieved from 2018 data.
coef_2018.groupby('State')= the mean average of impactful coefficients by state.

In [12]:
coef_df = pd.read_csv('./SNAP_data/2018_indicators/coef.csv')
coef_df['diff']= (coef_df['pred_0']-coef_df['pred_1']).abs()
coef_diff = coef_df.drop(columns=['pred_0','pred_1'])

In [13]:
coef_diff = coef_diff.sort_values(by=['diff'],ascending=False).reset_index(drop=True)
top = coef_diff.head(15)
top.to_csv('./SNAP_data/2018_indicators/top_impacts.csv',index=None)

In [15]:
coef_2018= sc.top_coef_df(top, df).set_index('state')
coef_2018.astype('int64').to_csv('./SNAP_data/2018_indicators/coef_2018_all.csv',index='state')

In [16]:
coef_2018.groupby('state').mean().astype('int64').to_csv('./SNAP_data/2018_indicators/coef_2018_state_avg.csv',index='state')